## Lendo o dataset

In [ ]:
from pathlib import Path

from projeto.dataset import read_dataset

base_path = Path.cwd()
df = read_dataset(base_path)

---

## Análise exploratória preliminar

Rode o código abaixo e responda:

- Todas as colunas estão de acordo com seus respectivos nomes?
- Existem dados faltantes?
- Para as variáveis contínuas: existem outliers ou anomalias?
- Para as variáveis categóricas: existem situações de desbalanceamento severo de classes?

***Respostas:***

1. Analisando os 5 primeiros exemplos mostrados pela função _describe_head() e os gráficos de distribuição gerados, todas features apresentam valores respectivos ao seu nome, tendo o detalhe de que as features capital.gain e capital.loss apresentam valores suspeitos 

2. Sim, há dados faltantes nas features workclass(1836), occupation(1843) e native.country(583), sendo todas elas features do tipo categóricas/qualitativas

3. Usando a função analisador_outlier_num()(Função que está no arquivo funcoes_extras.py) e analisando os gráficos obtidos por run_univariate_EDA(), foi avistado que as features/variáveis contínuas possuem outliers

4. Analisando os gráficos obtidos por run_univariate_EDA(), concluí que ocorre sim situações de desbalanceamento severo de classes, um exemplo é no gráfico de education-level distribuition, que mostra a categoria HS-Grade estando muito maior que outras categorias



---

In [ ]:
from projeto.eda import run_univariate_EDA

run_univariate_EDA(df)

## Filtragem e pré-processamento do dataset

Execute o código abaixo: 

In [ ]:
import pandas as pd


def cut_non_americans(df: pd.DataFrame) -> pd.DataFrame:
    df = df[df['native.country'] == 'United-States']
    df = df.drop(columns=['native.country'])
    return df

df = cut_non_americans(df)

Responda: qual o motivo para fazer este procedimento?

***Resposta***:

Por 'native.country' ser uma feature de pouca importância e influência, mas que possui dados faltantes, excluímos ela do dataset para evitar erros em relação à falta de dados e impactos negativos no processo de treino do modelo, além disso, o modelo pode acabar se tornando enviesado devido à frequência do número de exemplos com 'native.country' = 'United-States' ser muito maior em relação à outros, o que poderia causar overfitting, por isso, antes de excluir a coluna, mantemos os exemplos que possuem 'native.country' = 'United-States':

{'United-States': 29170, nan: 583, 'Mexico': 643, 'Greece': 29,

'Vietnam': 67, 'China': 75, 'Taiwan': 51, 'India': 100,

'Philippines': 198, 'Trinadad&Tobago': 19, 'Canada': 121,

'South': 80, 'Holand-Netherlands': 1, 'Puerto-Rico': 114,

'Poland': 60, 'Iran': 43, 'England': 90, 'Germany': 137, 'Italy': 73,

'Japan': 62, 'Hong': 20, 'Honduras': 13, 'Cuba': 95, 'Ireland': 24, 'Cambodia': 19,

'Peru': 31, 'Nicaragua': 34, 'Dominican-Republic': 70, 'Haiti': 44, 'El-Salvador': 106,

'Hungary': 13, 'Columbia': 59, 'Guatemala': 64, 'Jamaica': 81, 'Ecuador': 28,

'France': 29, 'Yugoslavia': 16, 'Scotland': 12, 'Portugal': 37, 'Laos': 18, 'Thailand': 18, 'Outlying-US(Guam-USVI-etc)': 14}

## Separação treino-teste

In [ ]:
X = df.drop(columns=['income'])
y = df['income']

In [ ]:
from projeto.config import RANDOM_SEED, TEST_SIZE
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=TEST_SIZE,
    random_state=RANDOM_SEED,
)

## Análise pós-separação treino-teste

Nem todos os pares de colunas merecem uma análise conjunta detalhada, isso rapidamente foge ao controle. Vamos privilegiar inicialmente:

- Análise descritiva:

    - Correlação entre *features* para identificar possiveis colinearidades simples e outras redundâncias de informação:

        - Entre *features* numéricas: correlação (e.g. Pearson)

        - Entre *features* categóricas: coeficiente de associação (e.g. [coeficiente V de Cramer](https://en.wikipedia.org/wiki/Cram%C3%A9r%27s_V))

        - Entre uma *feature* categórica e uma contínua: podemos *discretizar* a *feature* contínua por quantis e tratar esse problema como sendo um problema de associação entre categóricas, mas é meio gambiarra isso... paciencia. 

    - Correlação entre *target* e *features*, para identificar possíveis *features* que sejam:
    
        - Perfeitamente preditoras do target: isso é um sinal de que o *target* e a *feature* são a mesma coisa, e fazer um projeto de *machine learning* para prever $y$ à partir de... $y$ (!) é obviamente perda de tempo e dinheiro.

        - Totalmente não-correlacionadas com o target: *TALVEZ* indique que a *feature* em questão é inútil, mas isso nem sempre é verdade. Pode ser que a *feature* tem o seu valor preditivo apenas quando combinada com outras features. Pode ser que esta *feature* tenha um valor preditivo perfeito, e que a correlação se mostra baixa apenas porque a relação entre esta *feature* e o *target* é não-linear (e portanto não vai ser percebida pela correlação de Pearson).

- Visualização:

    - Ver os gráficos apropriados para a visualização conjunta de *target* e cada *feature*, e de pares de *features* que chamaram a atenção na análise descritiva.

Rode o código abaixo e responda as seguintes questões:

- Liste 3 *features* que parecem ter alta associação com o *target*
- O que acontece com as variáveis `capital.gain` e `capital.loss`? Explique. 

***Resposta***:

* Analisando os resultados obtidos pela função run_joint_EDA(), usando os dados de treino e os originais, as features com maior associação ao target(Income):
    - marital.status
    - education
    - relationship

* `capital.gain` e `capital.loss`

Ambas as fatures são descartadas devido à forma de como seus dados são distribuídos, algo que pode-se ver nos gráficos de distribuição de Capital Gain e Capital Loss acima, onde ambas as features são composta por 0 na maioria das vezes

Por causa disso, não usamos essas features, já que ao tentar fazer o modelo usá-las para obter o target, poderia acabar fazendo o modelo ficar muito enviesado, o que por consequência, causaria overfitting nas predições do modelo quando o modelo obtesse um valor que é diferente de 0

In [ ]:
from projeto.eda import run_joint_EDA

run_joint_EDA(X_train, y_train)
# run_joint_EDA(X, y)

# Gravando os arquivos de dados

Por fim, vamos gravar em disco os arquivos de dados resultantes da nossa análise, isso é fundamental!

In [ ]:
from projeto.dataset import save_processed_datasets

save_processed_datasets(
    X=X,
    y=y,
    X_train=X_train,
    X_test=X_test,
    y_train=y_train,
    y_test=y_test,
    base_path=base_path,
)

In [24]:
for data in df:
    print(data)

age
workclass
education
education.num
marital.status
occupation
relationship
race
sex
capital.gain
capital.loss
hours.per.week
income


Responda: o que aconteceria se ao invés de gravar estes arquivos, fizéssemos a leitura do arquivo original e uma simples separação treino-teste no *notebook* de modelagem, sem maiores cuidados?

***Resposta***:

Ao ter features irrelevantes com falta de dados ou dados incorretos/pouca variação, faria o modelo se tornar enviesado e instável, obtendo predições que poderiam causar overfitting/underfitting e/ou imprecisos/incorretos

Além de também não saber quais features o modelo teria que priorizar ao fazer a predição, um exemplo seria se um dos fatores decisivos do modelo fosse o 'native-country', ao ter essa feature como um fator decisivo, o modelo desconsideraria features com maior relevância ao target e acabaria por causar overfitting devido ao fato de que maioria dos dados do 'native-country' são 'United-States'